In [ ]:
!pip install keras==2.14.0

In [ ]:
from keras.datasets import imdb


In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

25000 train sequences
25000 test sequences


In [ ]:
print(X_train[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


the text of reviews is integer-encoded, where each integer represents a specific word in the dictionary.

# Decode reviews from index


In [ ]:
INDEX_FROM = 3
word_index = imdb.get_word_index()
word_index = {key:(value+INDEX_FROM) for key,value in word_index.items()}
word_index["<PAD>"] = 0    # the padding token
word_index["<START>"] = 1  # the starting token
word_index["<UNK>"] = 2    # the unknown token
reverse_word_index = {value:key for key, value in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(X_train[0])

1641221/1641221 [==============================] - 0s 0us/step


"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

In [ ]:
decode_review(X_test[0])

"<START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss"

consider the top 15,000 most common words. I will also consider 20% of the training set for validation purpose.

In [ ]:
vocab_size = 80000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words= vocab_size)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

25000 train sequences
25000 test sequences


Let us inspect how the first review looks like when we only consider the top 5,000 frequent words.

In [ ]:
decode_review(X_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

# Truncate and pad the review sequences
reviews can be different lengths. We will use the pad_sequences function to standardize the lengths of the reviews.



In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
maximum_sequence_length = 500 # maximum length of all review sequences

In [ ]:
X_train = pad_sequences(X_train, value= word_index["<PAD>"], padding= 'post', maxlen= maximum_sequence_length)
X_test = pad_sequences(X_test, value= word_index["<PAD>"], padding= 'post', maxlen= maximum_sequence_length)

print('X_train shape:', X_train.shape) # (n_samples, n_timesteps)
print('X_test shape:', X_test.shape)

X_train shape: (25000, 500)
X_test shape: (25000, 500)


In [ ]:
print(X_train[0])

[    1    14    22    16    43   530   973  1622  1385    65   458  4468
    66  3941     4   173    36   256     5    25   100    43   838   112
    50   670 22665     9    35   480   284     5   150     4   172   112
   167 21631   336   385    39     4   172  4536  1111    17   546    38
    13   447     4   192    50    16     6   147  2025    19    14    22
     4  1920  4613   469     4    22    71    87    12    16    43   530
    38    76    15    13  1247     4    22    17   515    17    12    16
   626    18 19193     5    62   386    12     8   316     8   106     5
     4  2223  5244    16   480    66  3785    33     4   130    12    16
    38   619     5    25   124    51    36   135    48    25  1415    33
     6    22    12   215    28    77    52     5    14   407    16    82
 10311     8     4   107   117  5952    15   256     4 31050     7  3766
     5   723    36    71    43   530   476    26   400   317    46     7
     4 12118  1029    13   104    88     4   381   

# Build the model
CNN

In [ ]:
!pip install scikeras


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import ParameterGrid
from keras.callbacks import EarlyStopping

In [ ]:
embedding_dim = 16

In [ ]:
def create_model(filters = 64, kernel_size = 3, strides=1, units = 256,
                 optimizer='adam', rate = 0.25, kernel_initializer ='glorot_uniform'):
  model = Sequential()
  # Embedding layer
  model.add(Embedding(vocab_size, embedding_dim, input_length= maximum_sequence_length))
    # Convolutional Layer(s)
  model.add(Dropout(rate))
  model.add(Conv1D(filters = filters, kernel_size = kernel_size, strides= strides,
                     padding='same', activation= 'relu'))
  model.add(GlobalMaxPooling1D())
    # Dense layer(s)
  model.add(Dense(units = units, activation= 'relu', kernel_initializer= kernel_initializer))
  model.add(Dropout(rate))
    # Output layer
  model.add(Dense(1, activation= 'sigmoid'))

    # Compile the model
  model.compile(loss='binary_crossentropy',
                  optimizer= optimizer,
                  metrics=['accuracy'])
  return model

In [ ]:
# Build the model
model = KerasClassifier(build_fn= create_model)

In [ ]:
def model_wrapper(filters, kernel_size, strides, units, optimizer, rate, kernel_initializer):
    def create_wrapped_model():
        return create_model(filters=filters, kernel_size=kernel_size, strides=strides,
                            units=units, optimizer=optimizer, rate=rate,
                            kernel_initializer=kernel_initializer)
    return create_wrapped_model

Tune hyperparameters

#  NOTE: Exhaustive Grid Search Maxes out Colab RAM. To circumvent this bottleneck, we use optuna, a hyperparameter tuning framework.

In [ ]:
### THE COMMENTED OUT PART IS THE CODE FOR EXHAUSTIVE GRID SEARCH

In [ ]:
# # Set the hyperparameters
# filters = [128] #[64, 128, 256]
# kernel_size = [5] #[3, 5, 7]
# strides= [1] # [1, 2, 5]
# Dense_units = [128, 512]
# kernel_initializer = ['TruncatedNormal'] #['zero', 'glorot_uniform', 'glorot_normal','TruncatedNormal']
# rate_dropouts = [0.25] #[0.1, 0.25, 0.5]
# optimizers = ['adam'] #['adam','rmsprop']
# epochs = [5]
# batches = [64] #[32, 64, 128]
# # ----------------------------------------------
# # Exhaustive Grid Search
# param_grid = dict(optimizer= optimizers, epochs= epochs, batch_size= batches,
#                   filters = filters, kernel_size = kernel_size, strides = strides,
#                   units = Dense_units, kernel_initializer= kernel_initializer, rate = rate_dropouts)

# grid = ParameterGrid(param_grid)
# param_sets = list(grid)

# param_scores = []
# for params in grid:

#     print(params)
#     model_params = {k: v for k, v in params.items() if k not in ['epochs', 'batch_size']}
#     wrapped_model = model_wrapper(**model_params)

#     model = KerasClassifier(build_fn=wrapped_model, epochs=params['epochs'],
#                             batch_size=params['batch_size'])

#     earlystopper = EarlyStopping(monitor='val_acc', patience= 0, verbose=1)

#     model.fit(X_train, y_train, shuffle=True, validation_data=(X_test, y_test), callbacks=[earlystopper])
#     history = model.model_.history



#     param_score = history.history['val_accuracy']
#     param_scores.append(param_score[-1])
#     print('+-'*50)
#     p = np.argmax(np.array(param_scores))

# print('param_scores:', param_scores)
# print("best score:", param_scores[p])
# # Choose best parameters
# best_params = param_sets[p]
# print("best parameter set", best_params)

In [ ]:
# def best_model():
#     return create_model(filters=best_params['filters'],
#                         kernel_size=best_params['kernel_size'],
#                         strides=best_params['strides'],
#                         units=best_params['units'],
#                         optimizer=best_params['optimizer'],
#                         rate=best_params['rate'],
#                         kernel_initializer=best_params['kernel_initializer'])


In [ ]:
# # Initialize KerasClassifier with the best model
# model = KerasClassifier(model=best_model, epochs=best_params['epochs'], batch_size=best_params['batch_size'])

# # Fit the model with the combined training and validation sets
# model.fit(np.vstack((X_train, X_test)), np.hstack((y_train, y_test)))

In [ ]:
# # Set the hyperparameters
# filters = [64] #[64, 128, 256]
# kernel_size = [5] #[3, 5, 7]
# strides= [1] # [1, 2, 5]
# Dense_units = [128, 512]
# kernel_initializer = ['glorot_normal'] #['zero', 'glorot_uniform', 'glorot_normal','TruncatedNormal']
# rate_dropouts = [0.25] #[0.1, 0.25, 0.5]
# optimizers = ['adam'] #['adam','rmsprop']
# epochs = [5]
# batches = [64] #[32, 64, 128]
# # ----------------------------------------------
# # Exhaustive Grid Search
# param_grid = dict(optimizer= optimizers, epochs= epochs, batch_size= batches,
#                   filters = filters, kernel_size = kernel_size, strides = strides,
#                   units = Dense_units, kernel_initializer= kernel_initializer, rate = rate_dropouts)

# grid = ParameterGrid(param_grid)
# param_sets = list(grid)

# param_scores = []
# for params in grid:

#     print(params)
#     model_params = {k: v for k, v in params.items() if k not in ['epochs', 'batch_size']}
#     wrapped_model = model_wrapper(**model_params)

#     model = KerasClassifier(build_fn=wrapped_model, epochs=params['epochs'],
#                             batch_size=params['batch_size'])

#     earlystopper = EarlyStopping(monitor='val_acc', patience= 0, verbose=1)

#     model.fit(X_train, y_train, shuffle=True, validation_data=(X_test, y_test), callbacks=[earlystopper])
#     history = model.model_.history



#     param_score = history.history['val_accuracy']
#     param_scores.append(param_score[-1])
#     print('+-'*50)
#     p = np.argmax(np.array(param_scores))

# print('param_scores:', param_scores)
# print("best score:", param_scores[p])
# # Choose best parameters
# best_params = param_sets[p]
# print("best parameter set", best_params)

In [ ]:
# def best_model():
#     return create_model(filters=best_params['filters'],
#                         kernel_size=best_params['kernel_size'],
#                         strides=best_params['strides'],
#                         units=best_params['units'],
#                         optimizer=best_params['optimizer'],
#                         rate=best_params['rate'],
#                         kernel_initializer=best_params['kernel_initializer'])


In [ ]:
# # Initialize KerasClassifier with the best model
# model = KerasClassifier(model=best_model, epochs=best_params['epochs'], batch_size=best_params['batch_size'])

# # Fit the model with the combined training and validation sets
# model.fit(np.vstack((X_train, X_test)), np.hstack((y_train, y_test)))

In [ ]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00


In [ ]:
import optuna
optuna.__version__

'3.4.0'

In [ ]:
def objective(trial):
    # Suggest hyperparameters
    filters = trial.suggest_categorical('filters', [64, 128, 256])
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5, 7])
    strides = trial.suggest_categorical('strides', [1, 2, 5])
    units = trial.suggest_categorical('units', [128, 512])
    kernel_initializer = trial.suggest_categorical('kernel_initializer', ['zero', 'glorot_uniform', 'glorot_normal', 'TruncatedNormal'])
    rate = trial.suggest_float('rate', 0.1, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    epochs = 5  # Fixed value in your setup
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    # Build and compile model
    wrapped_model = model_wrapper(filters, kernel_size, strides, units, optimizer, rate, kernel_initializer)
    model = KerasClassifier(build_fn=wrapped_model, epochs=epochs, batch_size=batch_size)

    # Early stopping
    earlystopper = EarlyStopping(monitor='val_accuracy', patience=0, verbose=1)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[earlystopper], verbose=0)
    history = model.model_.history

    # Retrieve the best validation accuracy
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy

# Running the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=80)  # Adjust the number of trials as needed

# Retrieve the best parameters
best_params = study.best_params
print("Best parameters:", best_params)



In [ ]:
# Build the best model
def best_model():
    return create_model(filters=best_params['filters'],
                        kernel_size=best_params['kernel_size'],
                        strides=best_params['strides'],
                        units=best_params['units'],
                        optimizer=best_params['optimizer'],
                        rate=best_params['rate'],
                        kernel_initializer=best_params['kernel_initializer'])